# reconnaissance de langue avec Azure Cognitive Service - Text Analytics







**colab-env** permet de récupérer mes credentials contenues dans le fichier vars.env sans les exposer dans le notebook

In [ ]:
!pip install colab-env --upgrade

  Created wheel for colab-env: filename=colab_env-0.2.0-py3-none-any.whl size=3836 sha256=3e0b232e39391fb231a7c25d529813c1fcecf6b56c2ef02cc643bedc872d172d
  Stored in directory: /root/.cache/pip/wheels/bb/ca/e8/3d25b6abb4ac719ecb9e837bb75f2a9b980430005fb12a9107
Successfully built colab-env


importation des librairies nécessaires

In [ ]:
import colab_env
import os
import requests
import json

Mounted at /content/gdrive


récupération de la clé API et de l'endpoint pour utiliser le service Cognitive Service - Text Analytics que j'ai créé sur Azure

In [ ]:
cog_key = os.getenv("COG_KEY")
cog_endpoint = os.getenv("COG_ENDPOINT")
language_recognition_endpoint = cog_endpoint + "/text/analytics/v3.1/languages"

**recognize_language** est la fonction que j'ai créée pour envoyer une requête **POST** à l'API pour la reconnaissance de langue

In [ ]:
def recognize_language(text, id, prettyPrint=False):
  payload = json.dumps({
      "documents": [
        {
            "id": id,
          "text": text
        }
      ]
  })
  headers = {
    'Ocp-Apim-Subscription-Key': cog_key,
    'Content-Type': 'application/json',
    'Accept': 'application/json'
  }

  response = requests.request("POST", language_recognition_endpoint, headers=headers, data=payload)
  if (prettyPrint == False):
    return response.text
  else:
    parsed = json.loads(response.text)
    return json.dumps(parsed, indent=4, sort_keys=True)

testons le rapidement avec une **entrée utilisateur**:

In [ ]:
text = input("write your text here : ")
lang = recognize_language(text, "1", True)
print(lang)

write your text here : ceci est un test
{
    "documents": [
        {
            "detectedLanguage": {
                "confidenceScore": 0.95,
                "iso6391Name": "fr",
                "name": "French"
            },
            "id": "1",
            "warnings": []
        }
    ],
    "errors": [],
    "modelVersion": "2021-01-05"
}


Pour faciliter les choses par la suite, nous utiliserons la classe **Language_Detection** que j'ai créée, et qui permet :
- d'envoyer plus d'un texte dans une seule requête (jusqu'à 1000 sont autorisés par Azure Text Analytics)
- de tronquer les textes trop longs pour être envoyés

In [ ]:
class Language_Detection:
    """
    Cette classe appelle le service d'analyse de texte d'Azure pour détecter la langue utilisée dans un texte.
    
    Attributes :
        COG_KEY: votre clé API (vous devez l'avoir écrite dans le fichier .env)
        COG_ENDPOINT: votre API endpoint (vous devez l'avoir écrite dans le fichier .env)
        LANGUAGE_RECOGNITION_URI
        HEADERS
        MAX_TXT_SIZE
        MAX_DOC_AMOUNT
    """
    
    def __init__(self):
        load_dotenv()
        self.COG_KEY = os.getenv('COG_KEY')
        self.COG_ENDPOINT = os.getenv('COG_ENDPOINT')
        self.LANGUAGE_RECOGNITION_URI = self.COG_ENDPOINT + "/text/analytics/v3.1/languages"
        self.HEADERS = {
          'Ocp-Apim-Subscription-Key': self.COG_KEY,
          'Content-Type': 'application/json',
          'Accept': 'application/json'
        }
        self.MAX_TXT_SIZE = 5120
        self.MAX_DOC_AMOUNT = 800
        
        
    def detect(self, text: str, id: str):
        '''Utilisez cette fonction pour détecter la langue d'un seul texte
        
        Paramètres :
            text (str):  Le texte dont vous voulez détecter la langue
            id (str):    L'identifiant du texte (si c'est juste pour un test et que votre texte n'a pas d'id, passez juste "1")
        
        Retour : 
            réponse de la requête (response.text)
        '''
        payload = json.dumps({
            "documents": [
              {
                  "id": id,
                "text": self.truncate_big_text(text)
              }
            ]
        })
        response = self.call_api(payload)
        return response
    
    
    def call_api(self, payload):
        '''Pour utiliser cette fonction, vous devez construire votre propre charge utile.
        veuillez noter que les limites sont :
        - 1000 documents par appel
        - 5120 caractères dans un seul document
        - 1 Mo par demande
        - 100 appels par minute

        Paramètres :
        payload (str) : string json avec la structure suivante :
                {
                    "documents": [
                        {"id": id1, "text": text 1},
                        {"id": id2, "text": text2},
                        ...
                        ]
                }
                
        Retour :
            réponse de la requête API (response.text)

        '''   
        response = requests.request("POST", self.LANGUAGE_RECOGNITION_URI, headers=self.HEADERS, data=payload)
        return response.text
    
    
    def truncate_big_text(self, text: str):
        '''Tronque le texte si sa longueur excède MAX_TXT_SIZE
        
        Paramètres:
            text (str): Le texte à tronquer
        '''
        if(len(text) > self.MAX_TXT_SIZE):
            text = text[:self.MAX_TXT_SIZE - 1]
        return text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/OC_Adrien_Godoy/Projet 1/data"

labels.csv  urls.txt	x_train.txt  y_train.txt
README.txt  x_test.txt	y_test.txt


In [ ]:
import pandas as pd

Liste des langues reconnues par **Azure Text Analytics** ([source](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/language-support?tabs=language-detection)) :

In [ ]:
langs = ["English", "Chinese", "Standard Chinese", "Literary Chinese", "Hindi", "Spanish", "French"]

Chemin des fichiers que nous utiliserons lors de nos tests.
Le dataset WiLI-2018 - Wikipedia Language Identification a été conçu comme un dataset de référence et contient 235 000 paragraphes dans 235 langues. Le dataset est équilibré et une répartition train-test est fournie.

Au lieu de travailler avec les 235 langues, je travaille avec 5 langues pour faciliter l'analyse.

In [ ]:
base_dir = "/content/drive/My Drive/OC_Adrien_Godoy/Projet 1/data"
labels_file = base_dir + "/labels.csv"
x_test_file = base_dir + "/x_test.txt"
y_test_file = base_dir + "/y_test.txt"

Construisons un DataFrame avec les données contenues dans le fichier "labels.csv", et regardons comment il se présente

In [ ]:
labels_df = pd.read_csv(labels_file, sep=";")
labels_df.head()

,Label,English,Wiki Code,ISO 369-3,German,Language family,Writing system,Remarks,Synonyms
0,ace,Achinese,ace,ace,Achinesisch,Austronesian,NaN,NaN,NaN
1,afr,Afrikaans,af,afr,Afrikaans,Indo-European,NaN,NaN,NaN
2,als,Alemannic German,als,gsw,Alemannisch,Indo-European,NaN,(ursprünglich nur Elsässisch),NaN
3,amh,Amharic,am,amh,Amharisch,Afro-Asiatic,NaN,NaN,NaN
4,ang,Old English,ang,ang,Altenglisch,Indo-European,NaN,(ca. 450-1100),Angelsächsisch


Récupère les labels de langue qui sont utilisés dans "y_train.txt" et "y_test.txt"

In [ ]:
lang_labels = list(labels_df[labels_df['English'].isin(langs)]['Label'])
lang_labels

['eng', 'fra', 'hin', 'lzh', 'spa', 'zho']

Récupère le nom complet des langues que nous utiliserons. C'est ce qui nous permettra de comparer le résultat des prédiction d'**Azure Text Analytics** avec ce qui est attendu.

In [ ]:
lang_names = (list(labels_df[labels_df['English'].isin(langs)]['English']))
lang_names

['English',
 'French',
 'Hindi',
 'Literary Chinese',
 'Spanish',
 'Standard Chinese']

Cette fonction extrait les paragraphes et des labels de langue pour les langues de la liste **langs**, et en fait un DataFrame:

In [ ]:
def read_file(x_file, y_file):
  # Récupère le contenu de  'y_file' dans un dataframe
  y_df = pd.read_csv(y_file, header=None)
  # y_df a une seule colonne qui s'appelera "labels"
  y_df.columns = ["Label"]

  # Récupère le contenu de 'x_file' dans une liste de strings
  with open(x_file, encoding="utf8") as f:
      x_pars = f.readlines()

  # Enlève les espaces et autres caractères invisibles (comme '\n') au début et à la fin des strings
  x_pars = [t.strip() for t in x_pars]
  # Convertit la liste en dataframe à une seule colonne: 'Paragraphe'
  x_df = pd.DataFrame(x_pars, columns=['Paragraph']) 
  # Ne garde que les paragraphes des langues connues (et enlève les autres)
  x_df = x_df[y_df['Label'].isin(lang_labels)]
  # Ne garde que ces langues dans lang_labels
  y_df = y_df[y_df['Label'].isin(lang_labels)]

  return (x_df, y_df)

Vérifions que le nombre de lignes correspond:

In [ ]:
lang_df, label_df = read_file(x_test_file, y_test_file)
lang_df.shape, label_df.shape

((3000, 1), (3000, 1))

In [ ]:
lang_df.head()

,Paragraph
42,大都会区有它自己的当地路边快餐口味，包括瓦达帕夫（蓬松面包劈开一半，填入锅贴）、潘尼普里（油...
50,La ciudad de San Cristóbal es sede del Hospita...
54,Les supporters de l'ASM Clermont Auvergne ont ...
57,"吳廷琰,越南首相，越南共和國總統也。成泰十三年誕於廣平。父廷可，宮監大臣。廷琰其三子也。年十..."
68,Anton (or Antonius) Maria Schyrleus (also Schy...


In [ ]:
label_df.head()

,Label
42,zho
50,spa
54,fra
57,lzh
68,eng


la fonction **verif_predictions()** va récupérer les paragraphes à tester, constituer un payload qui contiendra au maximum 1000 paragraphes, avec au plus 5120 caractères par paragraphe (limites Azure Cognitive Service - Text Analytics), et renvoyer un dataframe avec les colonnes suivantes :
- **id** : l'identifiant du texte
- **Paragraphe** : le paragraphe qui a été envoyé
- **expected** : la langue attendue
- **predicted** : la langue qui a été détectée
- **confidenceScore** : le score de confiance de la prédiction renvoyé par l'API
- **valid** : un booléen qui est **True** si **expected** et **predicted** ont la même valeur

In [ ]:
def verif_predictions():
    # structure du resultat des tests : id, paragraphe, expected, predicted, confidence score, valid
    lang_df, label_df = read_file(x_test_file, y_test_file)
    lang_df.index.name = "id"
    label_df.index.name = "id"
    last_index = lang_df.index[-1]
    max_doc_amount = Language_Detection().MAX_DOC_AMOUNT
    max_txt_size = Language_Detection().MAX_TXT_SIZE
    test_results = []
    doc_amount = 0
    payload = {}
    payload["documents"] = []
    
    for i in lang_df.index:
        if(doc_amount < max_doc_amount):
            doc_amount += 1
        # lang par est le paragraphe de langue qui a été extrait
        lang_par = lang_df["Paragraph"][i]
        if(len(lang_par) > max_txt_size):
            lang_par = lang_par[:max_txt_size - 1]
        document = {}
        document["id"] = i
        document["text"] = lang_par
        payload["documents"].append(document)
        if((doc_amount == max_doc_amount) or (i == last_index)):
            res = json.loads(Language_Detection().call_api(json.dumps(payload)))
            print(res)
            for document in res["documents"]:
                test_results.append([document["id"], 
                                 document["detectedLanguage"]["name"], 
                                 document["detectedLanguage"]["confidenceScore"]])
            payload = {}
            payload["documents"] = []
            doc_amount= 0
            if(i == last_index):
                df_results = pd.DataFrame(test_results, columns=["id", "Predicted", "confidenceScore"])
                df_results.set_index("id", inplace=True)
                df_results.index = df_results.index.astype(int)
                
                
                df_final = pd.merge(df_results, pd.merge(lang_df, label_df, left_index=True, right_index=True), left_index=True, right_index=True)
                df_final = pd.merge(df_final, labels_df, on="Label", how="left").set_index(df_final.index)
                df_final = df_final.drop(["Wiki Code", "ISO 369-3", "German", "Language family", "Writing system", "Remarks", "Synonyms", "Label"], axis=1)
                df_final = df_final.rename(columns={"English": "Expected"})
                df_final = df_final[["Paragraph", "Predicted", "Expected", "confidenceScore"]]
                # dans le dataset wili-2018, les langues chinoises n'ont pas le même nom
                # remplaçons donc les noms des langues de la colonne "Expected" par ceux correspondants pour les langues chinoises
                df_final.replace({"Standard Chinese": "Chinese Simplified"}, regex=True)
                df_final.replace({"Literary Chinese": "Chinese Traditional"}, regex=True)
                df_final["Valid"] = (df_final["Predicted"] == df_final["Expected"])
                return df_final

In [ ]:
df_results = verif_predictions()
df_results.to_csv(r'./results.csv')
dg_results.head()

In [ ]:
df_results.head()

Regardons quel est le pourcentage de bonnes et de mauvaises prédictions:

In [ ]:
df_results["Valid"].value_counts(normalize=True) * 100

regardons l'ensemble des prédictions erronées :

In [ ]:
df_results[df_results["Valid"] == False]

on peut aussi trier les résultats par langue :

In [2]:
df_results.groupby("Expected")["Valid"].value_counts(normalize=True) * 100

NameError: ignored

et afficher un histogramme des résultats par langue :

In [3]:
df_results["Valid"].hist(by=df_results["Expected"])

NameError: ignored

In [ ]:
df_results.groupby("Expected").count()["Valid"].hist() # produit le même résultat que la cellule précédente